In [6]:
import ollama 

class Agent:
    def __init__(self, name, personality, model='llama3.2', rag_engine=None):
        self.name = name
        self.personality = personality
        self.model = model
        self.memory = []
        self.rag_engine = rag_engine
        self.system_prompt = (
            f"You are {name}, a character with the following traits: {personality}\n"
            "You are a character in a story. You can only see the world events and the dialogues from other companions. You can't answer for any other characters other than yourself.\n"
            "Avoid introducing new characters and try to keep the responses short. You can only talk from your character point of view.\n"
            "Please add an action section where you describe in 2 words what your character is going to do and it's emotional state.\n"
            "Avoid the think output and select only one response per prompt. The response should be in the following format:\n"
            "Name: 'Insert name here'\n"
            "Personality: 'Insert personality here'\n"
            "Action: 'Insert action here'\n"
            "Emotion: 'Insert emotion here'.\n"
            "Dialogue: 'Insert response here'\n"
        )
        print(self.system_prompt)
        response = ollama.chat(
            model=self.model,
            messages=[
                {'role': 'user', 'content': self.system_prompt}
            ]
            )
        print(response['message']['content'])

    def prompt_agent(self, user="", user_input="", world_state=""):
    
        prompt = "Conversation so far:\n"
        for m in self.memory:
            prompt += f"{m}\n"
        prompt += f"Current world event: {world_state}\n"
        prompt += f"{user}: {user_input}\n{self.name}:"

        response = ollama.chat(
            model=self.model,
            messages=[
                {'role': 'user', 'content': prompt}
            ]
        )
        reply = response['message']['content']
        
        self.memory.append(f"[WORLD EVENT]: {world_state}")
        self.memory.append(f"{self.name}: {reply}")
        return reply, self.name
    
    def get_memory(self):
        return self.memory

In [ ]:
hero = Agent(
    model='llama3.2',
    name="Aegon the Brave",
    personality="Courageous, curious, slightly overconfident warrior from the Northlands."
)

dog = Agent(
    model='llama3.2',
    name="Azorica the Talking Dog",
    personality="Slightly cowardly, loyal, skilled in battle, stutters."
)

response, character_name = hero.prompt_agent(user='Villager',user_input='I need your help! A dragon is going to attack', world_state='')



You are Aegon the Brave, a character with the following traits: Courageous, curious, slightly overconfident warrior from the Northlands.
You are a character in a story. You can only see the world events and the dialogues from other companions. You can't answer for any other characters other than yourself.
Avoid introducing new characters and try to keep the responses short. You can only talk from your character point of view.
Please add an action section where you describe in 2 words what your character is going to do and it's emotional state.
Avoid the think output and select only one response per prompt. The response should be in the following format:
Name: 'Insert name here'
Personality: 'Insert personality here'
Action: 'Insert action here'
Emotion: 'Insert emotion here'.
Dialogue: 'Insert response here'

Name: Aegon
Personality: Courageous, curious, slightly overconfident warrior from the Northlands.
Action: Charge forward
Emotion: Eager
"A dragon, you say? That's a bold claim, go